# Dataset with features of a customer like nationality,gender,income,credit card,etc. We have to Predict that the person will leave the bank or Not

# Preprocessing

In [ ]:
#Importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#importing dataset
dataset=pd.read_csv('Churn_Modelling.csv')

X=dataset.iloc[:,3:13].values
Y=dataset.iloc[:,13].values

#changing the features like nationality,gender to fit in network
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
a=LabelEncoder()
X[:,1]=a.fit_transform(X[:,1])  #Converted 3 nationality to 0,1,2
X[:,2]=a.fit_transform(X[:,2])  #converted male,female to 0,1

b=OneHotEncoder(categorical_features=[1])  #Label encoder gave 0,1,2 value to nationality which will effect network so converting to OneHot
X=b.fit_transform(X).toarray()
X=X[:,1:]  # Trimming one column of oneHot value as two column are sufficient to represent 

#Splitting dataset in train and test
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=0)

#Scaling feature to same scale(0-1)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)




# Building ANN with 3 layers

In [ ]:
#importing Libraries
import keras
from keras.layers import Dense
from keras.models import Sequential
#Initialisng ANN model
model=Sequential()

#Setting up layers

#input layer
model.add(Dense(units=6,activation='relu',kernel_initializer='uniform',input_dim=11))
#hidden layer
model.add(Dense(units=6,activation='relu',kernel_initializer='uniform')) #Uniform weight initialisiation
#output layer
model.add(Dense(units=1,activation='sigmoid',kernel_initializer='uniform')) #Output layer having sigmoid for binary classifiers and softmax for categorical classifiers
#Compiling model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy']) #binary classifier->binary_crossentropy,categorical classifier->categorical_crossentropy
#Training step
model.fit(x_train,y_train,batch_size=30,epochs=100)

# Prediction

In [ ]:
#test set prediction
y_pred=model.predict(x_test)
y_pred=(y_pred>0.5)

# Predicting a single new observation
"""Predict if the customer with the following informations will leave the bank:
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000"""
res = model.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
res = (res > 0.5)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)

# k-Fold Cross Validation

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier  #using scikit-learn with keras
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense

#Function for Building deep net
def Build_Network():
  model=Sequential()
  
  model.add(Dense(units=6,activation='relu',kernel_initializer='uniform',input_dim=11))
  model.add(Dense(units=6,activation='relu',kernel_initializer='uniform'))
  model.add(Dense(units=1,activation='sigmoid',kernel_initializer='uniform'))
  
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  return model

model=KerasClassifier(build_fn=Build_Network,batch_size=10,epochs=100)
accuracies=cross_val_score(estimator=model,X=x_train,y=y_train,cv=10,n_jobs=-1) #10-fold cross validation using all cpu's
mean=accuracies.mean()
variance=accuracies.std()
print (mean)
print (variance)

# Tuning and Improving

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV  #for trying different combinations of hyperparameter
from keras.models import Sequential
from keras.layers import Dense

#Function for deepnet with optimizer as parameter
def Build_network(optimizer):
  model=Sequential()
  
  model.add(Dense(units=6,activation='relu',kernel_initializer='uniform',input_dim=11))

  model.add(Dense(units=6,activation='relu',kernel_initializer='uniform'))

  model.add(Dense(units=1,activation='sigmoid',kernel_initializer='uniform'))
  
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
  return model
model=KerasClassifier(build_fn=Build_network)
hyper_param={'batch_size':[25,32],'epochs':[100,200],'optimizer':['adam','rmsprop']} # Hypermeters for trying
grid_search=GridSearchCV(estimator=model,
                        param_grid=hyper_param,
                        scoring='accuracy',
                        cv=10) #10-fold coss validation with hperparameter tuning
grid_search=grid_search.fit(x_train,y_train)
best_param=grid_search.best_params_
best_acc=grid_search.best_score_

print(best_param)
print(best_acc)